In [43]:
import random
import pandas as pd
import numpy as np
import math
from operator import itemgetter
import argparse
np.random.seed(1024)

K=20
N=10
similarityMeasure="cosine"
data_file="../data/TopN-jupyter.txt"

In [44]:
data_fields = ['user_id', 'item_id']
data = pd.read_table(data_file, names=data_fields)
# 二维字典
train_data = {}

# 按照1:9划分数据集
for (user, item) in data.itertuples(index=False):
    train_data.setdefault(user,{})
    train_data[user][item] = 1

n_users = len(set(data['user_id'].values))
n_items = len(set(data['item_id'].values))


In [45]:
train_data

{'A': {'b': 1, 'd': 1},
 'B': {'a': 1, 'b': 1, 'c': 1},
 'C': {'a': 1, 'b': 1, 'd': 1},
 'D': {'a': 1, 'e': 1}}

In [46]:
# 建立item_user倒排表
# item->set
item_users = dict()
for u, items in train_data.items():
    for i in items:
        if i not in item_users:
            item_users[i] = set()
        item_users[i].add(u)
item_users

{'b': {'A', 'B', 'C'},
 'd': {'A', 'C'},
 'a': {'B', 'C', 'D'},
 'c': {'B'},
 'e': {'D'}}

In [50]:
 # 计算用户之间共同评分的物品数,C为修正后的，count为修正前的。
C = dict()
count = dict()
for i, users in item_users.items():
    for u in users:
        for v in users:
            if u == v:
                continue
            C.setdefault(u,{})
            C[u].setdefault(v,0)
            # 对热门物品进行惩罚
            C[u][v] += math.log(n_users/len(users))
            # 计算最终的用户相似度矩阵

            count.setdefault(u, {})
            count[u].setdefault(v, 0)
            count[u][v] += 1
user_sim = dict()
for u, related_users in C.items():
    user_sim[u]={}
    for v, cuv in related_users.items():
        if similarityMeasure == "cosine":
#             user_sim[u][v] = cuv / math.sqrt(len(train_data[u]) * len(train_data[v]))
            user_sim[u][v] = count[u][v] / math.sqrt(len(train_data[u]) * len(train_data[v]))
        else:
            user_sim[u][v] = count[u][v] / (len(train_data[u])+len(train_data[v])-count[u][v])
user_sim

{'A': {'B': 0.4082482904638631, 'C': 0.8164965809277261},
 'B': {'A': 0.4082482904638631,
  'C': 0.6666666666666666,
  'D': 0.4082482904638631},
 'C': {'A': 0.8164965809277261,
  'B': 0.6666666666666666,
  'D': 0.4082482904638631},
 'D': {'B': 0.4082482904638631, 'C': 0.4082482904638631}}

In [51]:
def predict(user):
    rank = dict()
    interacted_items = train_data[user]

    # 寻找最近的K个用户，利用它们的评分信息构造推荐列表
    for similar_user, similarity_factor in sorted(user_sim[user].items(),
                                                  key=itemgetter(1), reverse=True)[0:K]:
        for movie in train_data[similar_user]:
            if movie in interacted_items:
                continue
            rank.setdefault(movie, 0)
            # rank[movie] += similarity_factor*self.train_data[similar_user][movie]
            rank[movie] += similarity_factor

    rec_list=[]
    rec_items=sorted(rank.items(), key=itemgetter(1), reverse=True)[0:N]
    for item,score in rec_items:
        rec_list.append([item,score])
    return rec_list

In [52]:
for user in ['A','B','C','D']:
        rec_list=predict(user)
        print("给",user,"推荐：",rec_list)

给 A 推荐： [['a', 1.2247448713915892], ['c', 0.4082482904638631]]
给 B 推荐： [['d', 1.0749149571305296], ['e', 0.4082482904638631]]
给 C 推荐： [['c', 0.6666666666666666], ['e', 0.4082482904638631]]
给 D 推荐： [['b', 0.8164965809277261], ['c', 0.4082482904638631], ['d', 0.4082482904638631]]
